# Count Average Tokens Generated Per Experiment
This is approximated by counting tokens in the code generated by one arbitrary model bsed on the instructional prompt (the medium length promtp).

## Use this section if running in Google Colab

In [1]:
!pip install tiktoken

In [ ]:
!pip install -U datasets

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')             # Mount Google Drive to Colab session
!ls /content/drive/MyDrive                # Verify that your drive is accessible

# to be able to import local packages
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')

## Local code section

In [21]:
from helpers import read_problems, stream_jsonl
from prompts import complete_code_prompt, complete_task_prompt
from datasets import load_dataset

In [22]:
import tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")

def count_tokens(text):
    return len(enc.encode(text))

In [23]:
# gpt-4 latency (ms)
gpt4_latency_per_token = 0.018

## HumanEval Tokens

In [24]:
file = '/Users/andrew/Documents/01_documents/GWU_DENG/SEAS_8588_Praxis/2_Code/logs_results/final-round-one-model/HumanEval/OpenCodeInterpreter-DS-6.7B/HumanEval_OpenCodeInterpreter-DS-6.7B_complete_code_prompt_temperature1.0_topP1.0_completions_20250322_054821_9443.jsonl'
results = list( stream_jsonl(file) )
len(results), results[0]

(164,
 {'task_id': 'HumanEval/0',
  'completion': '```python\nfrom typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n\nprint(has_close_elements([1.0, 2.0, 3.0], 0.5))\nprint(has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3))\n```'})

In [25]:
# prompt template without content
prompt_template_tokens = count_tokens(complete_code_prompt)*len(results)
print(prompt_template_tokens)

# prompt content
dataset = load_dataset("openai/openai_humaneval")
prompt_content_tokens = count_tokens( ' '.join([i['prompt'] for i in dataset["test"]]) )
print(prompt_content_tokens)

# total prompt tokens (entire experiment)
total_prompt_tokens = prompt_template_tokens + prompt_content_tokens
print('Total tokens in prompts for the entire experiment:', total_prompt_tokens)

7708
21538
Total tokens in prompts for the entire experiment: 29246


In [26]:
# generated text
generated_tokens = sum([count_tokens(i['completion']) for i in results])
print('Total generated tokens for the entire experiment:', generated_tokens)

Total generated tokens for the entire experiment: 29054


In [27]:
# average latency
avg_tokens_per_one_generation  = generated_tokens/len(results)
print( avg_tokens_per_one_generation )
avg_latency_per_one_generation = avg_tokens_per_one_generation * gpt4_latency_per_token
print(f'Average latency per one code generation: {avg_latency_per_one_generation} seconds')

177.15853658536585
Average latency per one code generation: 3.188853658536585 seconds


## BigCodeBench Tokens

In [28]:
file = '/Users/andrew/Documents/01_documents/GWU_DENG/SEAS_8588_Praxis/2_Code/logs_results/final-round-one-model/BigCode/OpenCodeInterpreter-DS-6.7B/BigCode_OpenCodeInterpreter-DS-6.7B_complete_code_prompt_temperature1.0_topP1.0_completions_20250402_105939_8955.jsonl'
results = list( stream_jsonl(file) )
len(results), results[0]

(500,
 {'task_id': 'BigCodeBench/0',
  'completion': '```python\nimport itertools\nfrom random import shuffle\n\ndef task_func(numbers=list(range(1, 3))):\n    """\n    Calculates the average of the sums of absolute differences between each pair of consecutive numbers \n    for all permutations of a given list. Each permutation is shuffled before calculating the differences.\n\n    Args:\n    - numbers (list): A list of numbers. Default is numbers from 1 to 10.\n    \n    Returns:\n    float: The average of the sums of absolute differences for each shuffled permutation of the list.\n\n    Requirements:\n    - itertools\n    - random.shuffle\n\n    Example:\n    >>> result = task_func([1, 2, 3])\n    >>> isinstance(result, float)\n    True\n    """\n    permutations = list(itertools.permutations(numbers))\n    total_sum = 0\n    for perm in permutations:\n        shuffled_perm = list(perm)\n        shuffle(shuffled_perm)\n        diff_sum = 0\n        for i in range(len(shuffled_perm) -

In [29]:
# prompt template without content
prompt_template_tokens = count_tokens(complete_code_prompt)*len(results)
print(prompt_template_tokens)

# prompt content
file = 'data/Big_Code_Bench_Test.jsonl.gz'
tasks = read_problems(file)
prompt_content_tokens = count_tokens( ' '.join( [v['prompt'] for k,v in tasks.items()] ) )
print(prompt_content_tokens)

# total prompt tokens (entire experiment)
total_prompt_tokens = prompt_template_tokens + prompt_content_tokens
print('Total tokens in prompts for the entire experiment:', total_prompt_tokens)

23500
135165
Total tokens in prompts for the entire experiment: 158665


In [30]:
# generated text
generated_tokens = sum([count_tokens(i['completion']) for i in results])
print('Total generated tokens for the entire experiment:', generated_tokens)

Total generated tokens for the entire experiment: 159818


In [31]:
# average latency
avg_tokens_per_one_generation  = generated_tokens/len(results)
print( avg_tokens_per_one_generation )
avg_latency_per_one_generation = avg_tokens_per_one_generation * gpt4_latency_per_token
print(f'Average latency per one code generation: {avg_latency_per_one_generation} seconds')

319.636
Average latency per one code generation: 5.753448 seconds


## LBPP Tokens

In [32]:
file = '/Users/andrew/Documents/01_documents/GWU_DENG/SEAS_8588_Praxis/2_Code/logs_results/final-round-one-model/LBPP/OpenCodeInterpreter-DS-6.7B/LBPP_OpenCodeInterpreter-DS-6.7B_complete_task_prompt_temperature1.0_topP1.0_completions_20250405_005934_6995.jsonl'
results = list( stream_jsonl(file) )
len(results), results[0]

(162,
 {'task_id': 'lbpp/0',
  'completion': '```python\nimport numpy as np\n\ndef add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray:\n    avg_col = np.mean(ar, axis=1)\n    std_col = np.std(ar, axis=1)\n    return np.column_stack((ar, avg_col, std_col))\n\nar = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])\nprint(add_avg_and_std_cols_numpy(ar))\n\nar = np.array([[1], [1], [2]])\nprint(add_avg_and_std_cols_numpy(ar))\n\nar = np.array([[1, 2, 3, 4, 5]])\nprint(add_avg_and_std_cols_numpy(ar))\n```'})

In [33]:
# prompt template without content
prompt_template_tokens = count_tokens( complete_task_prompt )*len(results)
print(prompt_template_tokens)

# prompt content
file = 'data/LBPP_Test.jsonl.gz'
tasks = read_problems(file)
prompt_content_tokens1 = count_tokens( ' '.join( [v['prompt'] for k,v in tasks.items()] ) )
prompt_content_tokens2 = count_tokens( ' '.join( [v['test'] for k,v in tasks.items()] ) )    # adding tests
prompt_content_tokens = prompt_content_tokens1 + prompt_content_tokens2
print(prompt_content_tokens)

# total prompt tokens (entire experiment)
total_prompt_tokens = prompt_template_tokens + prompt_content_tokens
print('Total tokens in prompts for the entire experiment:', total_prompt_tokens)

11340
89246
Total tokens in prompts for the entire experiment: 100586


In [34]:
# generated text
generated_tokens = sum([count_tokens(i['completion']) for i in results])
print('Total generated tokens for the entire experiment:', generated_tokens)

Total generated tokens for the entire experiment: 85263


In [35]:
# average latency
avg_tokens_per_one_generation  = generated_tokens/len(results)
print( avg_tokens_per_one_generation )
avg_latency_per_one_generation = avg_tokens_per_one_generation * gpt4_latency_per_token
print(f'Average latency per one code generation: {avg_latency_per_one_generation} seconds')

526.3148148148148
Average latency per one code generation: 9.473666666666665 seconds


## MBPP Tokens

In [36]:
file = '/Users/andrew/Documents/01_documents/GWU_DENG/SEAS_8588_Praxis/2_Code/logs_results/final-round-one-model/MBPP/OpenCodeInterpreter-DS-6.7B/MBPP_OpenCodeInterpreter-DS-6.7B_complete_task_prompt_temperature1.0_topP1.0_completions_20250404_020543_0757.jsonl'
results = list( stream_jsonl(file) )
len(results), results[0]

(500,
 {'task_id': '11',
  'completion': '```python\ndef remove_Occ(s, c):\n    first_occurrence = s.find(c)\n    last_occurrence = s.rfind(c)\n    if first_occurrence == -1:\n        return s\n    elif first_occurrence == last_occurrence:\n        return s[:first_occurrence] + s[first_occurrence+1:]\n    else:\n        return s[:first_occurrence] + s[first_occurrence+1:last_occurrence] + s[last_occurrence+1:]\n\n# Test cases\nprint(remove_Occ("hello","l")) # Expected output: "heo"\nprint(remove_Occ("abcda","a")) # Expected output: "bcd"\nprint(remove_Occ("PHP","P")) # Expected output: "H"\n```'})

In [37]:
# prompt template without content
prompt_template_tokens = count_tokens( complete_task_prompt )*len(results)
print(prompt_template_tokens)

# prompt content
file = 'data/MBPP_Test.jsonl.gz'
tasks = read_problems(file)
prompt_content_tokens1 = count_tokens( ' '.join( [v['prompt'] for k,v in tasks.items()] ) )
prompt_content_tokens2 = count_tokens( ' '.join( [v['test'] for k,v in tasks.items()] ) )    # adding tests
prompt_content_tokens = prompt_content_tokens1 + prompt_content_tokens2
print(prompt_content_tokens)

# total prompt tokens (entire experiment)
total_prompt_tokens = prompt_template_tokens + prompt_content_tokens
print('Total tokens in prompts for the entire experiment:', total_prompt_tokens)

35000
50790
Total tokens in prompts for the entire experiment: 85790


In [38]:
# generated text
generated_tokens = sum([count_tokens(i['completion']) for i in results])
print('Total generated tokens for the entire experiment:', generated_tokens)

Total generated tokens for the entire experiment: 71265


In [39]:
# average latency
avg_tokens_per_one_generation  = generated_tokens/len(results)
print( avg_tokens_per_one_generation )
avg_latency_per_one_generation = avg_tokens_per_one_generation * gpt4_latency_per_token
print(f'Average latency per one code generation: {avg_latency_per_one_generation} seconds')

142.53
Average latency per one code generation: 2.56554 seconds
